# Préparation de l'espace de travail

## Importations

In [2]:
!pip install openpyxl cartiflette mapclassify

import openpyxl 
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import numpy as np
import seaborn as sns
from cartiflette import carti_download
import mapclassify
import plotly.express as px

## Lecture des bases de données

### Base de la démographie des médecins : medecin

In [ ]:
# Lecture du Excel medecins 
df_medecins_effectif_complet = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=1)
df_medecins_age_complet = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=2)
df_medecins_densite_complet = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=3)

In [ ]:
# Traitement de la feuille des effectifs : on ne conserve que les médecins généralistes et on enlève les variables pas pertinentes ou redondantes
df_medecins_effectif = df_medecins_effectif_complet[(df_medecins_effectif_complet['specialites_agregees'] == '1-Médecine générale') 
    & (df_medecins_effectif_complet['sexe'] == '0-Ensemble')
    & (df_medecins_effectif_complet['specialites'] == '00-Ensemble')
    ]

df_medecins_effectif.drop(axis = 1, columns = ['sexe', 'specialites', 'specialites_agregees'], inplace = True)
df_medecins_effectif.reset_index(drop = True, inplace = True)

df_medecins_effectif

In [ ]:
# Traitement de la feuille des âges moyens : on ne conserve que les médecins généralistes et on enlève les variables pas pertinentes ou redondantes
df_medecins_age = df_medecins_age_complet[(df_medecins_age_complet['specialites_agregees'] == '1-Médecine générale') 
    & (df_medecins_age_complet['sexe'] == '0-Ensemble')
    & (df_medecins_age_complet['specialites'] == '00-Ensemble')]

df_medecins_age.drop(axis = 1, columns = ['sexe', 'specialites', 'specialites_agregees'], inplace = True)
df_medecins_age.reset_index(drop = True, inplace = True)

df_medecins_age

In [ ]:
# Traitement de la feuille des densités : on ne conserve que les médecins généralistes et on enlève les variables pas pertinentes ou redondantes
df_medecins_densite = df_medecins_densite_complet[(df_medecins_densite_complet['specialites_agregees'] == '1-Médecine générale') 
    & (df_medecins_densite_complet['sexe'] == '0-Ensemble')
    & (df_medecins_densite_complet['specialites'] == '00-Ensemble')]

df_medecins_densite.drop(axis = 1, columns = ['sexe', 'specialites', 'specialites_agregees'], inplace = True)
df_medecins_densite.reset_index(drop = True, inplace = True)

df_medecins_densite

### Base de la prévalence de différentes maladies par groupe de population : santePublique

In [ ]:
# Lecture du csv santePublique
import zipfile

with zipfile.ZipFile("Bases de données/santePublique.zip", 'r') as zip_ref:
    zip_ref.extractall("Bases de données")

df_sante_publique = pd.read_csv('Bases de données/santePublique.csv', sep=";")

### Base de l'indicateur APL (Accessibilité Potentielle Localisée) pour les médecins généralistes 

In [ ]:
# Lecture du Excel APL_2022_2023 : tableau de la pondération de la population
# (consommation moyenne en soins de la tranche d'âge rapportée à la consommation moyenne de la population)
df_pond_population_2022_2023 = pd.read_excel("Bases de données/APL_2022_2023.xlsx", sheet_name=0)[21:40]
df_pond_population_2022_2023.reset_index(drop=True, inplace=True)

# Renommage des colonnes
df_pond_population_2022_2023.columns = ["Tranche d'âge", "Poids de la tranche d'âge en 2022", "Poids de la tranche d'âge en 2023"]

# Lecture du Excel APL_2015_2022 : tableau de pondération de la population (on exclut l'année 2022 en supposant que les données de APL_2022_2023 sont plus pertinentes)
df_pond_population_2015_2021 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=0)[21:40]
df_pond_population_2015_2021.reset_index(drop=True, inplace=True)

# Renommage des colonnes
df_pond_population_2015_2021.columns = ["Tranche d'âge", "Poids de la tranche d'âge en 2015", "Poids de la tranche d'âge en 2016", "Poids de la tranche d'âge en 2017", 
    "Poids de la tranche d'âge en 2018", "Poids de la tranche d'âge en 2019", "Poids de la tranche d'âge en 2021", "Poids de la tranche d'âge en 2022 bis"]
df_pond_population_2015_2021.drop("Poids de la tranche d'âge en 2022 bis", axis=1, inplace=True)

# Merging des deux tables sur la clé de la tranche d'âge
df_pond_population = df_pond_population_2015_2021.merge(right=df_pond_population_2022_2023, how="left", on=["Tranche d'âge"])

df_pond_population

In [ ]:
# Lecture des deux feuilles de la table APL_2022_2023
df_APL_2022 = pd.read_excel("Bases de données/APL_2022_2023.xlsx", sheet_name=1)[8:]
df_APL_2023 = pd.read_excel("Bases de données/APL_2022_2023.xlsx", sheet_name=2)[8:]

# Nettoyage et renommage des colonnes
bases = [df_APL_2022, df_APL_2023]
annee = 2022

for base in bases : 
    base.drop(8,inplace=True)
    base.reset_index(drop=True, inplace=True)
    base.columns = ["Code commune INSEE", "Commune", f"APL_{annee}", f"APL_{annee}_moins_65", f"APL_{annee}_moins_62", f"APL_{annee}_moins_60", f"population_standard_{annee-2}", f"population_totale_{annee-2}"]
    annee += 1

# Lecture des feuilles de la table APL_2015_2022 (on exclut l'année 2022 en supposant que les données de APL_2022_2023 sont plus pertinentes)
df_APL_2015 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=1)[8:]
df_APL_2016 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=2)[8:]
df_APL_2017 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=3)[8:]
df_APL_2018 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=4)[8:]
df_APL_2019 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=5)[8:]
df_APL_2021 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=6)[8:]

# Nettoyage et renommage des colonnes
bases = [df_APL_2015,df_APL_2016,df_APL_2017,df_APL_2018,df_APL_2019,df_APL_2021]
annee = 2015

for base in bases : 
    base.drop(8,inplace=True)
    base.reset_index(drop=True, inplace=True)
    base.columns = ["Code commune INSEE", "Commune", f"APL_{annee}", f"APL_{annee}_moins_65", f"population_standard_{annee-2}", f"population_totale_{annee-2}"]
    annee += 1
    if annee == 2020 : 
        annee += 1

bases.append(df_APL_2022)
df_APL = df_APL_2023
for base in bases : 
    df_APL = df_APL.merge(base, how='left', on=["Code commune INSEE", "Commune"])

df_APL['departement'] = df_APL['Code commune INSEE'].astype(str).str[:2]

In [ ]:
# On crée l'indicateur  APL par département pour l'année 2023
df_APL["APL_2023"] = pd.to_numeric(df_APL["APL_2023"], errors="coerce")

df_APL['APL_2023_pond'] = df_APL['APL_2023'] * df_APL['population_standard_2021']
df_APL["APL_dep_2023"] = df_APL.groupby("departement")['APL_2023_pond'].transform("mean")

df_APL["APL_dep_2023"] = pd.to_numeric(df_APL["APL_dep_2023"], errors="coerce")

### Base du nombre de médecins généralistes par commune (2025)

In [ ]:
df_freq_medecins_commune = pd.read_excel("Bases de données/medecins_commune_2025.xlsx")[4:]
df_freq_medecins_commune.reset_index(drop=True, inplace=True)
df_freq_medecins_commune.columns = ["Code commune INSEE", "Commune", "Médecins généralistes en 2025"]

df_freq_medecins_commune

### Base de la patientèle des médecins 

In [ ]:
df_patientele = pd.read_csv("Bases de données/Données_Patientele_Departementale.csv", sep=";")

# Enlever les caractères cachés dans le nom des colonnes (provoquaient des bugs dans la suite du code)
df_patientele.columns = df_patientele.columns.str.replace('\ufeff', '').str.strip()

df_patientele["nombre_patients_uniques"] = pd.to_numeric(df_patientele["nombre_patients_uniques"], errors="coerce")
df_patientele['profession_sante'].unique()

### Base d'indicateurs démographiques au niveau communal

In [16]:
df_pop_communes = pd.read_csv("Bases de données/Indicateurs_communale.csv", sep = ';')

df_pop_communes

/tmp/ipykernel_56095/2723403712.py:1: DtypeWarning: Columns (0,2,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pop_communes = pd.read_csv("Bases de données/Indicateurs_communale.csv", sep = ';')


,Insee - Statistiques locales,,.1,.2,.3,.4,.5,.6,.7,.8,...,.28,.29,.30,.31,.32,.33,.34,.35,.36,.37
0,Référentiel géographique : France par commune,,,,,,,,,,...,,,,,,,,,,
1,Code,Libellé,Population de 15 ans ou + selon le groupe soci...,Médiane du niveau de vie 2021,Police - Gendarmerie (en nombre) 2024,Supérette - Épicerie (en nombre) 2024,Boulangerie-pâtisserie (en nombre) 2024,"École maternelle, primaire, élémentaire (en no...",Collège (en nombre) 2024,Lycée (en nombre) 2024,...,Taux d'activité par tranche d'âge 2022,Taux d'activité par tranche d'âge 2022,Taux d'activité par tranche d'âge 2022,Part des pers. âgées de - 15 ans 2022,Part des pers. âgées de - de 25 ans 2022,Part des pers. âgées de 25 à 64 ans 2022,Part des pers. âgées de 75 ans ou + 2022,Part des familles avec 1 enf. de - de 25 ans 2022,Part des familles avec 2 enf. de - de 25 ans 2022,Part des familles avec 3 enf. ou plus de - de ...
2,01001,L'Abergement-Clémenciat,39,25820,0,0,0,1,0,0,...,45.5,97.6,61.8,18.9,26.5,53.4,8.8,25.5,18.2,7.3
3,01002,L'Abergement-de-Varey,5,24480,0,0,0,0,0,0,...,20,97.4,61.5,20.9,30,51.6,8.4,18.7,31.3,6.2
4,01004,Ambérieu-en-Bugey,392,21660,1,5,14,7,2,2,...,51.8,90.1,62.1,19.2,32.6,49.3,8.8,21.9,18.7,12.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34872,97613,M'Tsangamouji,N/A - résultat non disponible,N/A - résultat non disponible,0,43,5,9,1,0,...,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible
34873,97614,Ouangani,N/A - résultat non disponible,N/A - résultat non disponible,0,71,0,9,1,2,...,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible
34874,97615,Pamandzi,N/A - résultat non disponible,N/A - résultat non disponible,1,51,6,9,1,1,...,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible
34875,97616,Sada,N/A - résultat non disponible,N/A - résultat non disponible,1,51,10,12,1,1,...,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible,N/A - résultat non disponible


# Premières visualisations des bases

## Base de la démographie des médecins

### Evolution du nombre de généralistes en France

In [ ]:
# Définition de la base de données réduite 
df_cond = df_medecins_effectif[(df_medecins_effectif['exercice']=='0-Ensemble') 
    & (df_medecins_effectif['tranche_age']=='00-Ensemble') 
    & (df_medecins_effectif['region'] == '00-Ensemble') 
    & (df_medecins_effectif['territoire'] == "0-France entière")] 
df = df_cond.copy()

# Définition des abscisses : années 2012 à 2025 
annees = list(range(2012,2026)) 

# Définition des ordonnées : effectifs des années 2012 à 2025 
effectifs = df[[f"effectif_{a}" for a in annees]].values.flatten() 
plt.plot(annees, effectifs, marker = "o") 

# Affichage du graphique 
plt.ylabel("Nombre de médecins généralistes") 
plt.title("Évolution du nombre de médecins généralistes en France") 

# Sous-texte
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins généralistes en France, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.26, "Lecture : Le nombre de médecins généralistes en France est passé d'environ 101 500 en 2012 à 100 000 en 2025.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.grid(True)
plt.show()

### Evolution du nombre de généralistes par région

In [ ]:
# Définition de la base de données réduite 
df_cond = df_medecins_effectif[(df_medecins_effectif['exercice']=='0-Ensemble') & (df_medecins_effectif['tranche_age']=='00-Ensemble') & (df_medecins_effectif['region'] != '00-Ensemble')]
df = df_cond.copy()

# Définition des abscisses : années 2012 à 2025 
annees = list(range(2012,2026))

# Définition des ordonnées : effectifs par région des années 2012 à 2025 
for region in df['region'].unique() :
    df_region = df[(df['region'] == region) & (df['departement'] == '000-Ensemble')]
    effectifs = df_region[[f"effectif_{a}" for a in annees]].values.flatten()
    plt.plot(annees, effectifs, label = region, marker = "o")

# Affichage du graphique 
plt.xlabel("Année")
plt.ylabel("Nombre de médecins généralistes")
plt.title(f"Évolution du nombre de médecins généralistes par région")

# Légende
plt.legend(title="Région", bbox_to_anchor=(1.05, 1), loc="upper left")

# Sous-texte
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins généralistes par région en France, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.26, "Lecture : Le nombre de médecins généralistes en Île de France est passé d'environ 18 500 en 2012 à 16 000 en 2025.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.grid(True)
plt.show()

### Evolution de l'âge moyen des généralistes en France

In [ ]:
# Définition de la base de données réduite 
df_cond = df_medecins_age[(df_medecins_age['exercice']=='0-Ensemble') 
    & (df_medecins_age['region'] == '00-Ensemble') 
    & (df_medecins_age['territoire'] == "0-France entière")] 
df = df_cond.copy()

# Définition des abscisses : années 2012 à 2025 
annees = list(range(2012,2026)) 


# Définition des ordonnées : effectifs des années 2012 à 2025 
effectifs = df[[f"am_{a}" for a in annees]].values.flatten() 
plt.plot(annees, effectifs, marker = "o") 

# Affichage du graphique plt.xlabel("Année") 
plt.ylabel("Âge moyen des médecins généralistes") 
plt.title("Évolution de l'âge moyen des médecins généralistes en France") 
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins généralistes en France, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.26, "Lecture : L'âge moyen des médecins généralistes en France est passé de 51,1 ans en 2012 à 50,4 ans en 2025.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.grid(True)
plt.show()

### Evolution du nombre de médecins généralistes par tranche d'âge en France

In [ ]:
# Définition de la base de données réduite 
df_cond = df_medecins_effectif[(df_medecins_effectif['exercice']=='0-Ensemble') 
    & (df_medecins_effectif['region'] == '00-Ensemble') 
    & (df_medecins_effectif['territoire'] == "0-France entière")
    & (df_medecins_effectif['tranche_age'] != "00-Ensemble")] 
df = df_cond.copy()

# Définition des abscisses : années 2012 à 2025 
annees = list(range(2012,2026)) 

# Définition des ordonnées : effectifs par tranche des années 2012 à 2025 
for tranche in df['tranche_age'].unique() : 
    df_age = df[df['tranche_age'] == tranche]
    effectifs = df_age[[f"effectif_{a}" for a in annees]].values.flatten() 
    plt.plot(annees, effectifs, label = tranche, marker = "o") 

# Affichage du graphique 
plt.ylabel("Nombre de médecins généralistes") 
plt.title("Évolution du nombre de médecins généralistes par tranche d'âge en France") 

# Légende
plt.legend(title="Tranche d'âge", bbox_to_anchor=(1.05, 1), loc="upper left")

# Sous-texte
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins généralistes en France, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.26, "Lecture : Le nombre de médecins généralistes entre 55 et 59 ans en France est passé d'environ 19 000 en 2012 à 10 000 en 2025.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.grid(True)
plt.show()

### Evolution de la proportion des médecins généralistes par tranche d'âge en France

In [ ]:
# Définition de la base de données réduite 
df_cond = df_medecins_effectif[(df_medecins_effectif['exercice']=='0-Ensemble') 
    & (df_medecins_effectif['region'] == '00-Ensemble') 
    & (df_medecins_effectif['territoire'] == "0-France entière")] 
df = df_cond.copy()

# Définition des abscisses : années 2012 à 2025 
annees = list(range(2012,2026)) 

# Définition de nouvelles variables proportion par année
for annee in annees:
    total = df.loc[df['tranche_age'] == '00-Ensemble', f'effectif_{annee}'].values[0]
    df[f'proportion_{annee}'] = df[f'effectif_{annee}'] / total

# Définition des nuances de couleurs
cmap = cm.get_cmap("OrRd")
colors = cmap(np.linspace(0, 1, 12))
i = 0

# Définition des ordonnées : proportion des tranches des années 2012 à 2025 
for tranche in df['tranche_age'].unique() : 
    if tranche != "00-Ensemble" :
        df_age = df[df['tranche_age'] == tranche]
        proportion = df_age[[f"proportion_{a}" for a in annees]].values.flatten()
        plt.plot(annees, proportion, label = tranche, marker = "o", color = colors[i]) 
        i += 1

# Affichage du graphique 
plt.ylabel("Proportion des médecins généralistes") 
plt.title("Évolution de la proportion de médecins généralistes par tranche d'âge en France") 

# Légende
plt.legend(title="Tranche d'âge", bbox_to_anchor=(1.05, 1), loc="upper left")

# Sous-texte
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins généralistes en France, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.26, "Lecture : La proportion de médecins généralistes entre 30 et 34 ans en France est passée de 0.06 en 2012 à 0.125 en 2025.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.grid(True)
plt.show()

### Etude de la densité de médecin par département pour l'année 2023

In [ ]:
df = df_medecins_densite.copy()
df = df[(df['departement'] != '000-Ensemble') 
    & (df['tranche_age'] == '00-Ensemble')
    & (df['exercice'] == '0-Ensemble')]

df.reset_index(inplace = True, drop = True)

df['departement'] = df['departement'].astype(str).str[:3]

# Départements

departements = carti_download(
    values = ["France"],
    crs = 4326,
    borders = "DEPARTEMENT",
    vectorfile_format="geojson",
    simplification=50,
    filter_by="FRANCE_ENTIERE_DROM_RAPPROCHES",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

departements['INSEE_DEP'] = departements['INSEE_DEP'].str.zfill(3)

# Test d'affichage de la carte
departements.plot().axis('off')

In [ ]:
# On joint les deux bases
departements = departements.merge(df, left_on = "INSEE_DEP", right_on = "departement", how="left")
departements = departements.to_crs(2154)

departements

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

departements.plot(
    ax=ax,
    column="densite_2023",
    cmap="OrRd",               
    linewidth=0,
    edgecolor="lightgrey",
    legend=True,
)

ax.axis("off")
ax.set_title("Densité de médecins par département en 2023", fontsize=14)

plt.show()

## Base de l'indicateur APL

### Etude de l'indicateur APL par commune/département pour l'année 2023

In [ ]:
df_APL['APL_2023'].dropna().describe()
df_APL['APL_dep_2023'].dropna().describe()

### Cartographie de l'indicateur APL par commune pour l'année 2023

In [ ]:
df = df_APL.copy()
departements = df_APL['departement'].unique()

# 1. Communes
communes = carti_download(
    values = departements,
    crs = 4326,
    borders="COMMUNE_ARRONDISSEMENT",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

# 2. Départements 
departements = communes.dissolve("INSEE_DEP")

communes = communes.merge(df, left_on = "INSEE_COG", right_on = "Code commune INSEE", how="left")
communes = communes.to_crs(2154)

In [ ]:
communes.plot()

In [ ]:
# Réalisation de la carte
fig, ax = plt.subplots(figsize=(10,10))

# On choisit le nombre de quantiles : 
n = 5

communes.plot(
    ax=ax,
    column="APL_2023",
    cmap="OrRd",               
    linewidth=0,
    edgecolor="lightgrey",
    legend=True,
    scheme = "quantiles",
    k = n, 
    legend_kwds={"labels": [i for i in range(n)]},
    missing_kwds={"color": "lightgrey", "label": "Données manquantes"}
)

ax.axis("off")
ax.set_title("Indicateur APL par commune en 2023", fontsize=14)

q = mapclassify.Quantiles(communes['APL_2023'].dropna(), k=n)
mapping = {i: s for i, s in enumerate(q.get_legend_classes())}
def replace_legend_items(legend, mapping):
    for txt in legend.texts:
        for k, v in mapping.items():
            if txt.get_text() == str(k):
                txt.set_text(v)
replace_legend_items(ax.get_legend(), mapping)

plt.show()

### Création de l'indicateur APL par département

In [ ]:
# Départements

departements = carti_download(
    values = ["France"],
    crs = 4326,
    borders = "DEPARTEMENT",
    vectorfile_format="geojson",
    simplification=50,
    filter_by="FRANCE_ENTIERE_DROM_RAPPROCHES",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

# Test d'affichage de la carte
departements.plot().axis('off')

In [ ]:
# On crée une copie de la table
df = df_APL.copy()

# On refait une carte, cette fois-ci des départements
df.drop_duplicates('departement', inplace = True)
departements = departements.merge(df, left_on = "INSEE_DEP", right_on = "departement", how="left")
departements = departements.to_crs(2154)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

departements.plot(
    ax=ax,
    column="APL_dep_2023",
    cmap="OrRd",               
    linewidth=0,
    edgecolor="lightgrey",
    legend=True,
    scheme = "quantiles",
    k = 10, 
    legend_kwds={"labels": [i for i in range(10)]},
    missing_kwds={"color": "lightgrey", "label": "Données manquantes"}
)

ax.axis("off")
ax.set_title("Indicateur APL par département en 2023", fontsize=14)

q10 = mapclassify.Quantiles(departements['APL_dep_2023'].dropna(), k=10)
mapping = {i: s for i, s in enumerate(q10.get_legend_classes())}
def replace_legend_items(legend, mapping):
    for txt in legend.texts:
        for k, v in mapping.items():
            if txt.get_text() == str(k):
                txt.set_text(v)
replace_legend_items(ax.get_legend(), mapping)

plt.show()

In [ ]:
departements["APL_dep_2023_log"] = np.log1p(departements["APL_dep_2023"])

fig, ax = plt.subplots(figsize=(10,10))

departements.plot(
    ax=ax,
    column="APL_dep_2023_log",
    cmap="OrRd",               
    linewidth=0,
    legend=True,
    edgecolor="lightgrey", 
)

ax.axis("off")
ax.set_title("Indicateur APL logarithmique par département en 2023", fontsize=14)

plt.show()

## Base de la patientèle (nombre de patients uniques par médecin) - pas pertinent, base trop petite ?

### Etude du nombre moyen de patients par médecin en 2017

In [ ]:
# On s'intéresse d'abord uniquement à l'année 2017
df = df_patientele.copy()
df = df[(df['annee'] == 2017) & (df['profession_sante'] == 'Ensemble des médecins généralistes')]

df['nombre_patients_uniques']

In [ ]:
# Départements
departements = carti_download(
    values = ["France"],
    crs = 4326,
    borders = "DEPARTEMENT",
    vectorfile_format="geojson",
    simplification=50,
    filter_by="FRANCE_ENTIERE_DROM_RAPPROCHES",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

# Test d'affichage de la carte
departements.plot().axis('off')
departements['INSEE_DEP'].unique()

In [ ]:
# On refait une carte, cette fois-ci des départements

df.drop_duplicates('departement', inplace = True)
departements = departements.merge(df, left_on = "INSEE_DEP", right_on = "departement", how="left")
departements = departements.to_crs(2154)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

departements.plot(
    ax=ax,
    column="nombre_patients_uniques",
    cmap="OrRd",               
    linewidth=0.05,
    legend=True,
    edgecolor="lightgrey", 
)

ax.axis("off")
ax.set_title("Nombre moyen de patiens par médecin en 2017", fontsize=14)

plt.show()

## Corrélation entre APL et différentes variables

### Premier test uniquement sur l'année 2023

In [ ]:
# On réarrange les tables APL densités de médecins
df1 = df_APL.copy()
df1['departement'] = df1['departement'].str.zfill(3)
df.drop_duplicates('departement', inplace = True)

df2 = df_medecins_densite.copy()
df2 = df2[(df2['departement'] != '000-Ensemble') 
    & (df2['tranche_age'] == '00-Ensemble')
    & (df2['exercice'] == '0-Ensemble')]
df2['departement'] = df2['departement'].astype(str).str[:3]

df = df1.merge(df2, on = 'departement')
df.reset_index(drop = True, inplace = True)

df['APL_dep_2023']

In [ ]:
sns.regplot(
    data = df,
    x = 'densite_2023',
    y = 'APL_dep_2023',
    fit_reg = True,
    scatter_kws={'alpha':0.5}
)

### On étend la régression aux années pour lesquelles on a des données : 2015 à 2023, 2020 exclu

In [ ]:
# On réarrange les tables APL densités de médecins
df1 = df_APL.copy()
df1['departement'] = df1['departement'].str.zfill(3)

df2 = df_medecins_densite.copy()
df2 = df2[(df2['departement'] != '000-Ensemble') 
    & (df2['tranche_age'] == '00-Ensemble')
    & (df2['exercice'] == '0-Ensemble')]
df2['departement'] = df2['departement'].astype(str).str[:3]

# On crée l'indicateur  APL par département pour les années 2015 à 2022, 2020 exclu
annees = [2015, 2016, 2017, 2018, 2019, 2021, 2022]

for annee in annees : 
    df1[f"APL_{annee}"] = pd.to_numeric(df1[f"APL_{annee}"], errors="coerce")
    df1[f'APL_{annee}_pond'] = df1[f'APL_{annee}'] * df1[f'population_standard_{annee - 2}']

    df1[f"APL_dep_{annee}"] = df1.groupby("departement")[f'APL_{annee}_pond'].transform("mean")
    df1[f"APL_dep_{annee}"] = pd.to_numeric(df1[f"APL_dep_{annee}"], errors="coerce")

df1.drop_duplicates('departement', inplace = True)

In [ ]:
# On transforme les tables larges en tables longues de manière à pouvoir faire la régression en utilisant les données de toutes les années
annees = [2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023]
APL = [f"APL_dep_{annee}" for annee in annees]
densite = [f"densite_{annee}" for annee in annees] 

# Pour l'APL
df1_long = df1.melt(
    id_vars = ['departement'], 
    value_vars = APL, 
    var_name = 'annee',
    value_name = 'APL'
)

df1_long['annee'] = df1_long['annee'].astype(str).str[8:]

# Pour la densité
df2_long = df2.melt(
    id_vars = ['departement'], 
    value_vars = densite, 
    var_name = 'annee',
    value_name = 'densite' 
)

df2_long['annee'] = df2_long['annee'].astype(str).str[8:]

# On joint les deux tables
df = pd.merge(df2_long, df1_long, on=['departement', 'annee'])
df['APL'] = np.log(df['APL'])

In [ ]:
# Régression utilisant les données toutes années confondues
sns.regplot(
    data = df,
    x = 'densite',
    y = 'APL',
    fit_reg = True,
    scatter_kws={'alpha':0.5},
    color="lightblue", line_kws=dict(color="orange")
)

### Carte interactive pour l'évolution de l'APL par commune

In [ ]:
# On transforme la tables large en table longue 
annees = [2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023]
df1 = df_APL.copy()
APL = [f"APL_{annee}" for annee in annees]

# Pour l'APL
df1_long = df1.melt(
    id_vars = ['Code commune INSEE'], 
    value_vars = APL, 
    var_name = 'annee',
    value_name = 'APL'
)

df1_long['annee'] = df1_long['annee'].astype(str).str[4:]

df1_long

In [ ]:
fig = px.choropleth(
    df1_long, 
    geojson=communes, # Contour des départements
    locations='Code commune INSEE', 
    color='APL',
    animation_frame='annee', # C'est ici que la magie opère
    range_color=[0, 4], # Fixer l'échelle pour que les couleurs soient comparables
    scope="europe" 
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()